In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

#用Sobel计算帧图像上点在两个方向上的梯度

def compute_spatial_gradients(image):
    I_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    I_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    return I_x, I_y

#用土相减法计算帧间的时间梯度

def compute_temporal_gradient(prev_frame, current_frame):
    I_t = current_frame.astype(float) - prev_frame.astype(float)
    return I_t


# H-S光流算法
def horn_schunck(prev_frame, current_frame, alpha=1.0, num_iterations=100):
    I_x, I_y = compute_spatial_gradients(current_frame)
    I_t = compute_temporal_gradient(prev_frame, current_frame)
    
    u = np.zeros(current_frame.shape)
    v = np.zeros(current_frame.shape)
    
    kernel = np.array([[1/12, 1/6, 1/12],
                       [1/6,  0,  1/6],
                       [1/12, 1/6, 1/12]], float)
    
    print("Starting Horn-Schunck iterations...")
    for i in tqdm(range(num_iterations), desc="Horn-Schunck Iterations"):
        u_avg = cv2.filter2D(u, -1, kernel)
        v_avg = cv2.filter2D(v, -1, kernel)
        
        term = (I_x * u_avg + I_y * v_avg + I_t) / (alpha**2 + I_x**2 + I_y**2)
        u = u_avg - I_x * term
        v = v_avg - I_y * term
    
    return u, v

def draw_optical_flow(img, u, v, step=16):
    h, w = img.shape
    y, x = np.mgrid[step//2:h:step, step//2:w:step].astype(int)
    
    fx, fy = u[y, x], v[y, x]
    
    # 调整维度，确保 lines 的维度为 (N, 2, 2)
    lines = np.stack((x.flatten(), y.flatten(), (x + fx).flatten(), (y + fy).flatten()), axis=1)
    lines = lines.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    
    vis = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    
    # 修改 cv2.arrowedLine 的参数
    for i in range(lines.shape[0]):
        (x1, y1), (x2, y2) = lines[i]
        cv2.arrowedLine(vis, (x1, y1), (x2, y2), (0, 255, 0), 1, tipLength=0.3)
    
    return vis

def process_video(video_path, output_path, alpha=1.0, num_iterations=100):
    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    if not ret:
        print("Error: unable to read video file.")
        return

    prev_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_height, frame_width = prev_frame.shape

    # 创建视频编写器
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (frame_width, frame_height))

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"Processing video with {frame_count} frames...")
    for frame_idx in tqdm(range(frame_count - 1), desc="Video Frames"):
        ret, frame = cap.read()
        if not ret:
            break
        
        current_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        u, v = horn_schunck(prev_frame, current_frame, alpha, num_iterations)
        
        flow_vis = draw_optical_flow(current_frame, u, v)
        
        # 写入视频帧
        out.write(flow_vis)
        
        prev_frame = current_frame
    
    cap.release()
    out.release()
    cv2.destroyAllWindows()

# 调用主函数
process_video('trafic_cut.mp4', 'output_optical_flow.mp4')

Processing video with 15 frames...


Video Frames:   0%|          | 0/14 [00:00<?, ?it/s]

Starting Horn-Schunck iterations...



Video Frames:   7%|▋         | 1/14 [00:08<01:53,  8.70s/it]

Starting Horn-Schunck iterations...


...


Video Frames:  86%|████████▌ | 12/14 [01:42<00:16,  8.39s/it]

Starting Horn-Schunck iterations...



Video Frames:  93%|█████████▎| 13/14 [01:51<00:08,  8.43s/it]

Starting Horn-Schunck iterations...



Video Frames: 100%|██████████| 14/14 [01:59<00:00,  8.55s/it]
